In [5]:
# load packages

import time

from IPython import get_ipython
from IPython.display import clear_output
import os
import matplotlib.pyplot as plt
import numpy as np
from scipy.sparse import csc_matrix
from scipy import signal as sg
import scipy
import pickle

from tifffile.tifffile import imwrite,imread
from tqdm.auto import tqdm,trange

from copy import deepcopy
import h5py

import caiman as cm
from caiman.source_extraction.cnmf import cnmf,params
from caiman.paths import caiman_datadir
from caiman.utils.visualization import get_contours

try:
    if __IPYTHON__:
        get_ipython().run_line_magic('load_ext', 'autoreload')
        get_ipython().run_line_magic('autoreload', '2')
except NameError:
    pass

def load_pickle(file_path):
    """
    Load a dictionary from a pickle file.

    Args:
    - file_path (str): Path to the pickle file.

    Returns:
    - dict: Loaded dictionary.
    """
    with open(file_path, 'rb') as f:
        data = pickle.load(f)
    return data

def flatten_extend(matrix):
     flat_list = []
     for row in matrix:
         flat_list.extend(row)
     return flat_list

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
# only slice movies are going through this - so mov_means is always 52
data_dir_l = [#r'\\nasquatch\data\2p\peter\slice\20230822_SL2PL52\slice\preprocessing\20230822_SL2PL52_slice_maddy_done_qc_done_m',
             r'\\nasquatch\data\2p\peter\slice\20230822_SL2PL52\wells001\preprocessing\20230822_SL2PL52_wells001_maddy_done_qc_doneqc_m',
             r'\\nasquatch\data\2p\peter\slice\20230822_SL3PL58\slice\preprocessing\20230822_SL3PL58_slice_Done_qc_doneqc_m']
             # r'\\nasquatch\data\2p\peter\slice\20230824_SL1PL60\slice\preprocessing\20230824_SL1PL60_slice_charlotte_done_qc_doneqc_m',
             # r'\\nasquatch\data\2p\peter\slice\20230824_SL1PL60\wells001\preprocessing\20230824_SL1PL60_wells001_charlotte_done_qc_doneqc_m',
             # r'\\nasquatch\data\2p\peter\slice\20230824_SL1PL60\wells101\preprocessing\20230824_SL1PL60_wells101_charlotte_done_qc_doneqc_m',
             # r'\\nasquatch\data\2p\peter\slice\20230824_SL2PL54\slice\preprocessing\20230824_SL2PL54_slice_Done_qc_doneqc_m',
             # r'\\nasquatch\data\2p\peter\slice\20230829_SL1PL56\slice\preprocessing\20230829_SL1PL56_slice_Tiara_done_qc_done_m',
             # r'\\nasquatch\data\2p\peter\slice\20230829_SL1PL56\wells001\preprocessing\20230829_SL1PL56_wells001_done_qc_done_m',
             # r'\\nasquatch\data\2p\peter\slice\20230829_SL2PL39\slice\preprocessing\20230829_SL2PL39_slice_Done_qc_doneqc_m',
             # r'\\nasquatch\data\2p\peter\slice\20230829_SL3PL61\slice\preprocessing\20230829_SL3PL61_slice_Done_qc_doneqc_m',
             # r'\\nasquatch\data\2p\peter\slice\20230830_SL1PL28\slice\preprocessing\20230830_SL1PL28_slice_maddy_done_qc_done_m',
             # r'\\nasquatch\data\2p\peter\slice\20230830_SL1PL28\wells001\preprocessing\20230830_SL1PL28_wells001_Done_qc_doneqc_m',
             # r'\\nasquatch\data\2p\peter\slice\20230830_SL2PL60\slice\preprocessing\20230830_SL2PL60_slice_dONE_qc_doneqc_m',
             # r'\\nasquatch\data\2p\peter\slice\20230831_SL1PL55\slice\preprocessing\20230831_SL1PL55_slice_Yael_Done_qc_done_m',
             # r'\\nasquatch\data\2p\peter\slice\20230831_SL1PL55\wells001\preprocessing\20230831_SL1PL55_wells001_Yael_Done_qc_doneqc_m']
mmap_dir_l = [#r'D:\pkalugin\memmap_cache\SL2PL52_230822_001_noartifact_wch1_aniso4_scale4AFS2PL4_toref28_wxy_nobg',
             r'D:\pkalugin\memmap_cache\PL52_230822_001_full_wch0_aniso4_scale4AFS2PL4_toref52_wxy_nobg',
             r'D:\pkalugin\memmap_cache\SL3PL58_230822_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref28_wxy_nobg']
             # r'D:\pkalugin\memmap_cache\SL1PL60_230824_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref22_wxy_nobg',
             # r'D:\pkalugin\memmap_cache\PL60_230824_001_full_wch1_aniso4_scale4AFS2PL4_toref18_wxy_nobg',
             # r'D:\pkalugin\memmap_cache\PL60_230824_101_full_wch0_aniso4_scale4AFS2PL4_toref20_wxy_nobg',
             # r'D:\pkalugin\memmap_cache\SL2PL54_230824_001_noartifact_wch1_aniso4_scale4AFS2PL4_toref33_wxy_nobg',
             # r'D:\pkalugin\memmap_cache\SL1PL56_230829_001_noartifact_wch1_aniso4_scale4AFS2PL4_toref28_wxy_nobg',
             # r'D:\pkalugin\memmap_cache\PL56_230829_001_full_wch1_aniso4_scale4AFS2PL4_toref52_wxy_nobg',
             # r'D:\pkalugin\memmap_cache\SL2PL39_230829_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref28_wxy_nobg',
             # r'D:\pkalugin\memmap_cache\SL3PL61_230829_001_noartifact_wch1_aniso4_scale4AFS2PL4_toref28_wxy_nobg',
             # r'D:\pkalugin\memmap_cache\SL1PL28_230830_001_noartifact_wch1_aniso4_scale4AFS2PL4_toref28_wxy_nobg',
             # r'D:\pkalugin\memmap_cache\PL28_230830_001_full_wch0_aniso4_scale4AFS2PL4_toref52_wxy_nobg',
             # r'D:\pkalugin\memmap_cache\SL2PL60_230830_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref28_wxy_nobg',
             # r'D:\pkalugin\memmap_cache\SL1PL55_230831_001_noartifact_wch1_aniso4_scale4AFS2PL4_toref28_wxy_nobg',
             # r'D:\pkalugin\memmap_cache\PL55_230831_001_full_wch0_aniso4_scale4AFS2PL4_toref52_wxy_nobg']
mov_means = 52

doraw = True
n_cores = 30
nrg_thr = 0.95

# Parameters (unlikely to change)
n_range_lim = 10 # size of n_range below which SNR considered unreliable
Athresh = 0.05 # overlap threshold - automatically split anything below it
cr_thresh = 0.9 # component-raw correlation threshold below which component deemed suspicious quality
pb_thresh = 0.95 # component-best parent correlation threshold above which component deemed likely merge

In [2]:
# only slice movies are going through this - so mov_means is always 52
data_dir_l = [r'\\nasquatch\data\2p\peter\slice\20230509_DA1PL53\slice\preprocessing\20230509_DA1PL53_slice_Yael_Done_qc_done_m',
             r'\\nasquatch\data\2p\peter\slice\20230509_DA1PL53\wells101\preprocessing\20230509_DA1PL53_wells101_Yael_Done_qc_doneqc_m',
             r'\\nasquatch\data\2p\peter\slice\20230509_DA2PL55\slice\preprocessing\20230509_DA2PL55_slice_Yael_done_qc_doenqc_m',
             r'\\nasquatch\data\2p\peter\slice\20230511_DA1PL37\slice\preprocessing\20230511_DA1PL37_slice_david_done_qc_doneqc_badcomponents_m']
             #r'\\nasquatch\data\2p\peter\slice\20230511_DA2PL45\slice\preprocessing\20230511_DA2PL45_slice_Yael_done_qc_doneqc_m',
             #r'\\nasquatch\data\2p\peter\slice\20230606_SL1PL35\slice\preprocessing\20230606_SL1PL35_slice_maddy_done_qc_done_m',
             #r'\\nasquatch\data\2p\peter\slice\20230607_SL1PL47\slice\preprocessing\20230607_SL1PL47_slice_Done_qc_doneqc_m',
             #r'\\nasquatch\data\2p\peter\slice\20230607_SL1PL47\wells001\preprocessing\20230607_SL1PL47_wells001_Done_qc_doneqc_m',
             #r'\\nasquatch\data\2p\peter\slice\20230607_SL2PL42\slice\preprocessing\20230607_SL2PL42_slice_Yael_done_qc_doneqc_m',
             #r'\\nasquatch\data\2p\peter\slice\20230608_SL1PL37\slice\preprocessing\20230608_SL1PL37_slice_Tiara_done_qc_doneqc_m',
             #r'\\nasquatch\data\2p\peter\slice\20230608_SL2PL45\slice\preprocessing\20230608_SL2PL45_slice_Tiara_done_qc_doneqc_m',
             #r'\\nasquatch\data\2p\peter\slice\20230616_SL2PL39\slice\preprocessing\20230616_SL2PL39_slice_Tiara_done_qc_doneqc_m',
             #r'\\nasquatch\data\2p\peter\slice\20230727_SL1PL52\slice\preprocessing\20230727_SL1PL52_slice_Tiara_done_qc_doneqc_m',
             #r'\\nasquatch\data\2p\peter\slice\20230727_SL2PL32\slice\preprocessing\20230727_SL2PL32_slice_charlotte_done_qc_doneqc_m',
             #r'\\nasquatch\data\2p\peter\slice\20230727_SL2PL32\wells101\preprocessing\20230727_SL2PL32_wells101_charlotte_done_qc_doneqc_m',
             #r'\\nasquatch\data\2p\peter\slice\20230801_SL1PL35\slice\preprocessing\20230801_SL1PL35_slice_maddy_done_qc_done_m',
             #r'\\nasquatch\data\2p\peter\slice\20230801_SL1PL35\wells101\preprocessing\20230801_SL1PL35_wells101_maddy_done_qc_doneqc_m',
             #r'\\nasquatch\data\2p\peter\slice\20230801_SL2PL34\slice\preprocessing\20230801_SL2PL34_slice_Tiara_done_qc_doneqc_m',
             #r'\\nasquatch\data\2p\peter\slice\20230817_SL1PL55\slice\preprocessing\20230817_SL1PL55_slice_maddy_done_qc_done_m',
             #r'\\nasquatch\data\2p\peter\slice\20230817_SL1PL55\wells001\preprocessing\20230817_SL1PL55_wells001_maddy_done_qc_done_m',
             #r'\\nasquatch\data\2p\peter\slice\20230817_SL1PL55\wells101\preprocessing\20230817_SL1PL55_wells101_maddy_done_qc_done_m',
             #r'\\nasquatch\data\2p\peter\slice\20230817_SL2PL60\slice\preprocessing\20230817_SL2PL60_slice_Tiara_done_qc_doneqc_m',
             #r'\\nasquatch\data\2p\peter\slice\20230822_SL1PL32\slice\preprocessing\20230822_SL1PL32_slice_Done_qc_doneqc_m']
mmap_dir_l = [r'D:\pkalugin\memmap_cache\DA1PL53_230509_001_wch0_aniso4_scale4AFS2PL4_toref10_wxy_nobg',
             r'D:\pkalugin\memmap_cache\PL53_230509_101_full_wch0_aniso4_scale4.4AFS2PL4_toref8_wxy_nobg',
             r'D:\pkalugin\memmap_cache\DA2PL55_230509_001_wch0_aniso4_scale4AFS2PL4_toref16_wxy_nobg',
             r'D:\pkalugin\memmap_cache\xx0DA1PL37_230511_001_wch0_aniso4_scale4AFS2PL4_toref15_wxy_nobg']
             # r'D:\pkalugin\memmap_cache\xx0DA2PL45_230511_001_wch0_aniso4_scale4AFS2PL4_toref18_wxy_nobg',
             # r'D:\pkalugin\memmap_cache\SL1PL35_230606_001_wch0_aniso4_scale4.4AFS2PL4_toref18_wxy_nobg',
             # r'D:\pkalugin\memmap_cache\SL1PL47_230607_001_wch0_aniso4_scale4.4AFS2PL4_toref15_wxy_nobg',
             # r'D:\pkalugin\memmap_cache\PL47_230607_001_full_wch0_aniso4_scale4.4AFS2PL4_toref27_wxy_nobg',
             # r'D:\pkalugin\memmap_cache\SL2PL42_230607_001_noartifact_wch0_aniso4_scale4.4AFS2PL4_toref25_wxy_nobg',
             # r'D:\pkalugin\memmap_cache\SL1PL37_230608_001_noartifact_wch0_aniso4_scale4.4AFS2PL4_toref18_wxy_nobg',
             # r'D:\pkalugin\memmap_cache\SL2PL45_230608_001_noartifact_wch0_aniso4_scale4.4AFS2PL4_toref26_wxy_nobg',
             # r'D:\pkalugin\memmap_cache\SL2PL39_230616_001_wch0_aniso4_scale4.4AFS2PL4_toref18_wxy_nobg',
             # r'D:\pkalugin\memmap_cache\SL1PL52_230727_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref18_wxy_nobg',
             # r'D:\pkalugin\memmap_cache\SL2PL32_230727_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref25_wxy_nobg',
             # r'D:\pkalugin\memmap_cache\PL32_230727_101_full_wch0_aniso4_scale4AFS2PL4_toref39_wxy_nobg',
             # r'D:\pkalugin\memmap_cache\SL1PL35_230801_001_noartifact_wch0_aniso4_scale4.4AFS2PL4_toref27_wxy_nobg',
             # r'D:\pkalugin\memmap_cache\PL35_230801_101_full_wch0_aniso4_scale4AFS2PL4_toref76_wxy_nobg',
             # r'D:\pkalugin\memmap_cache\SL2PL34_230801_001_noartifact_wch0_aniso4_scale4.4AFS2PL4_toref27_wxy_nobg',
             # r'D:\pkalugin\memmap_cache\SL1PL55_230817_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref27_wxy_nobg',
             # r'D:\pkalugin\memmap_cache\PL55_230817_001_full_wch0_aniso4_scale4AFS2PL4_toref18_wxy_nobg',
             # r'D:\pkalugin\memmap_cache\PL55_230817_101_full_wch0_aniso4_scale4AFS2PL4_toref39_wxy_nobg',
             # r'D:\pkalugin\memmap_cache\SL2PL60_230817_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref27_wxy_nobg',
             # r'D:\pkalugin\memmap_cache\SL1PL32_230822_001_noartifact_wch1_aniso4_scale4AFS2PL4_toref28_wxy_nobg']
mov_means = 52

doraw = True
n_cores = 30
nrg_thr = 0.95

# Parameters (unlikely to change)
n_range_lim = 10 # size of n_range below which SNR considered unreliable
Athresh = 0.05 # overlap threshold - automatically split anything below it
cr_thresh = 0.9 # component-raw correlation threshold below which component deemed suspicious quality
pb_thresh = 0.95 # component-best parent correlation threshold above which component deemed likely merge

In [ ]:
# the overall flow here is to load the original CNMF, load original vars_dict and merge points
# then make vs don't make the merges based on the points
# and finally run the refit followed by the raw refit for both the original and the merged components (so keep the option of unmerging some later)

In [9]:
for m in trange(0,len(data_dir_l)):
    data_dir = data_dir_l[m]
    mmap_dir = mmap_dir_l[m]
    
    data_dir = data_dir.replace(f'\\',f'/')
    mmap_dir = mmap_dir.replace(f'\\',f'/')
    
    ## Loading all the inputs
    os.chdir(data_dir)
    cnmf_path = os.path.join(data_dir, 'ch0_means_movie_nobg_cnmf.hdf5')
    
    # CNMFE model
    cnmf_model = cnmf.load_CNMF(cnmf_path, 
                                n_processes=1,
                                dview=None)
    
    cnmf_path_merged = os.path.join(data_dir, 'ch0_means_movie_nobg_cnmf_merged_refit.hdf5')
    
    # CNMFE model
    cnmf_model_merged_refit = cnmf.load_CNMF(cnmf_path_merged, 
                                n_processes=1,
                                dview=None)
    
    # Line up static inputs
    spcomps = np.reshape(cnmf_model_merged_refit.estimates.A.toarray(),cnmf_model_merged_refit.estimates.dims + (-1,),order='F')
    spcomps = spcomps.transpose([3,2,0,1]).astype(np.float32)
    CY = cnmf_model_merged_refit.estimates.C + cnmf_model_merged_refit.estimates.YrA # temporal loadings
    
    spcomps_e = np.concatenate((np.zeros(spcomps[0,...][np.newaxis,...].shape,dtype=np.float32),spcomps))
    spcomps_e2 = np.argmax(spcomps_e,axis=0)
    
    spcomps_l = np.zeros(spcomps.shape).astype('int')
    for ncomp in range(spcomps.shape[0]):
        spcomps_l[ncomp,...] = spcomps[ncomp,...]>0
    
    mc_memmapped_fname = [i for i in os.listdir() if 'memmap__' in i][0]
    Yr, dims, T = cm.load_memmap(mc_memmapped_fname)
    images = np.reshape(Yr.T, [T] + list(dims), order='F')

    save_path = 'ch0_means_movie_nobg_compfilt.pickle'
    
    if save_path in os.listdir(data_dir):
        vars_dict1 = load_pickle(os.path.join(data_dir, save_path))

    merge_path = f'ch0_means_movie_nobg_compmerge.npy'
    if merge_path in os.listdir(data_dir):
        merge_dict1 = np.load(os.path.join(data_dir, merge_path),allow_pickle=True)[()]

    qc_spotcomps = [[]]*len(merge_dict1['MpointsC'])
    for m in range(len(merge_dict1['MpointsC'])):
        qc_spotcomps2 = []
        qc_spotcomps3 = []
        for r in range(len(merge_dict1['MpointsC'][m])):
            qc_spotcomps2 = qc_spotcomps2 + np.where(spcomps[:,merge_dict1['MpointsC'][m][r,0],merge_dict1['MpointsC'][m][r,1],merge_dict1['MpointsC'][m][r,2]]!=0)[0].tolist()
        tr = [qc_spotcomps3.append(x) for x in qc_spotcomps2 if x not in qc_spotcomps3]
        qc_spotcomps[m] = qc_spotcomps3

    merge_sets = []
    for g in range(len(qc_spotcomps)):
        merge_sets.append([g] + qc_spotcomps[g])

    groups_list2 = list()
    for x in merge_sets:
        s = x
        for y in groups_list2[:]:
            if len(set(s) & set(y)) != 0:
                s += y
                groups_list2.remove(y)
        groups_list2.append(s)

    groups_list = []
    for g in range(len(groups_list2)):
        gl = []
        tr = [gl.append(x) for x in groups_list2[g] if x not in gl]
        groups_list.append(gl)

    vars_dict1_m = deepcopy(vars_dict1)
    
    sm = []
    for g in groups_list:
        y = []
        for i in g:
            y += vars_dict1['SMG'][i]
        sm.append(y)
    vars_dict1_m['SMG'] = sm

    #%% start a cluster for parallel processing (if a cluster already exists it will be closed and a new session will be opened)
    n_processes = n_cores
    if 'dview' in locals():
        cm.stop_server(dview=dview)
    c, dview, n_processes = cm.cluster.setup_cluster(
        backend='multiprocessing', n_processes=n_processes, single_thread=False)
    
    parameter_dict = {#'fnames': cnmf_model.params.data['fnames'][0].decode('UTF-8'),
                      #'dims': cnmf_model.params.data['dims'],
                      'p': cnmf_model.params.temporal['p'],
                      'nb': cnmf_model.params.temporal['nb'],
                      'rf': cnmf_model.params.patch['rf'],
                      'K': cnmf_model.params.init['K'], 
                      'gSig': cnmf_model.params.init['gSig'],
                      'gSiz': cnmf_model.params.init['gSiz'],
                      'stride': cnmf_model.params.patch['stride'],
                      'method_init': cnmf_model.params.init['method_init'],
                      'rolling_sum': cnmf_model.params.init['rolling_sum'], #important
                      'rolling_length': cnmf_model.params.init['rolling_length'],
                      'only_init': cnmf_model.params.patch['only_init'], #also keep an eye on this
                      'ssub': cnmf_model.params.init['ssub'],
                      'tsub': cnmf_model.params.init['tsub'],
                      'merge_thr': cnmf_model.params.merging['merge_thr'], 
                      'bas_nonneg': cnmf_model.params.temporal['bas_nonneg'],
                      'kernel': cnmf_model.params.init['kernel'],
                      'nIter': cnmf_model.params.init['nIter']
                     }
    
    parameters = params.CNMFParams(params_dict=parameter_dict) # CNMFParams is the parameters class

    cnmf_model_merged_m = cnmf.CNMF(n_processes, 
                           params=parameters, 
                           dview=dview)
    cnmf_model_merged_m.params.data['dims'] = cnmf_model.params.data['dims']
    cnmf_model_merged_m.params.spatial['n_pixels_per_process'] = cnmf_model.params.spatial['n_pixels_per_process']
    cnmf_model_merged_m.params.preprocess['n_pixels_per_process'] = cnmf_model.params.preprocess['n_pixels_per_process']
    cnmf_model_merged_m.params.spatial['nrgthr'] = nrg_thr
    cnmf_model_merged_m.mmap_file = cnmf_model.mmap_file
    cnmf_model_merged_m.estimates = deepcopy(cnmf_model.estimates)
    
    cnmf_model_merged_m.estimates.idx_components = flatten_extend(vars_dict1_m["SMG"])
    cnmf_model_merged_m.estimates.idx_components_bad = np.setdiff1d(range(cnmf_model.estimates.C.shape[0]),flatten_extend(vars_dict1_m["SMG"]))
    cnmf_model_merged_m.estimates.manual_merge(vars_dict1_m["SMG"],cnmf_model.params)
    cnmf_model_merged_m.estimates.select_components(use_object=True)
    
    print(cnmf_model.estimates.C.shape)
    print(cnmf_model_merged_refit.estimates.C.shape)
    print(cnmf_model_merged_m.estimates.C.shape)
    
    Yr = np.transpose(np.reshape(images, (T, -1), order='F'))
    Yr.filename = images.filename
    Yr = cnmf_model_merged_m.preprocess(Yr)
    cnmf_model_merged_m.update_temporal(Yr, use_init=False)
    cnmf_model_merged_m.estimates.normalize_components()
    print(cnmf_model_merged_m.estimates.C.shape)
    
    save_path_merged_m = os.path.join(data_dir, f'ch0_means_movie_nobg_cnmf_merged_m.hdf5')
    cnmf_model_merged_m.save(save_path_merged_m)
    print("Saved merged model")
    
    #~2 hrs on colab05, XX on beastmode, 3 hrs on megatron
    cnmf_model_merged_m_refit = cnmf_model_merged_m.refit(images)
    print(cnmf_model_merged_m_refit.estimates.C.shape)
    
    # save merged and refit outputs
    #print("Saving CNMF model...")
    
    save_path_merged_m_refit = os.path.join(data_dir, f'ch0_means_movie_nobg_cnmf_merged_m_refit.hdf5')
    cnmf_model_merged_m_refit.save(save_path_merged_m_refit)
    print("Saved refit model")

    if doraw:
        # apply merged/refit model to raw movie
        fname_mmap = mmap_dir + f'/' + os.listdir(mmap_dir)[0]
        Yr2, dims2, T2 = cm.load_memmap(fname_mmap)
        images2 = np.reshape(Yr2.T, [T2] + list(dims2), order='F')
        
        K = cnmf_model_merged_refit.estimates.C.shape[0]
        T = cnmf_model_merged_refit.estimates.C.shape[1]*mov_means
        Cin = cm.source_extraction.cnmf.initialization.resize(cnmf_model_merged_refit.estimates.C, [K,T])
        if Cin.shape[1] < images2.shape[0]:
            Cin = np.concatenate((Cin,np.repeat(Cin[...,Cin.shape[1]-1][:,np.newaxis],images2.shape[0]-Cin.shape[1],axis=1)),axis=1,dtype=Cin.dtype)    
        YrAin = cm.source_extraction.cnmf.initialization.resize(cnmf_model_merged_refit.estimates.YrA, [K,T])
        if YrAin.shape[1] < images2.shape[0]:
            YrAin = np.concatenate((YrAin,np.repeat(YrAin[...,YrAin.shape[1]-1][:,np.newaxis],images2.shape[0]-YrAin.shape[1],axis=1)),axis=1,dtype=YrAin.dtype)  
        
        cnmf_model_mr_raw = cnmf.CNMF(n_processes, 
                               params=parameters, 
                               dview=dview)
        cnmf_model_mr_raw.estimates = deepcopy(cnmf_model_merged_refit.estimates)
        cnmf_model_mr_raw.estimates.C = Cin
        cnmf_model_mr_raw.estimates.YrA = YrAin
        cnmf_model_mr_raw.estimates.b = None
        cnmf_model_mr_raw.estimates.f = None
        cnmf_model_mr_raw.mmap_file = cnmf_model.mmap_file
        
        #~2.5 hrs on colab05, XX on beastmode, 5.5 hrs on megatron
        cnmf_model_mr_raw_refit = cnmf_model_mr_raw.refit(images2)
        print(cnmf_model_mr_raw_refit.estimates.C.shape)
        
        # save merged and refit outputs
        #print("Saving CNMF model...")
        save_path_mr_raw = os.path.join(data_dir, f'ch0_means_movie_nobg_cnmf_mr_raw.hdf5')
        save_path_mr_raw_refit = os.path.join(data_dir, f'ch0_means_movie_nobg_cnmf_mr_raw_refit.hdf5')
        cnmf_model_mr_raw.save(save_path_mr_raw)
        cnmf_model_mr_raw_refit.save(save_path_mr_raw_refit)
    
        K_m = cnmf_model_merged_m_refit.estimates.C.shape[0]
        T_m = cnmf_model_merged_m_refit.estimates.C.shape[1]*mov_means
        Cin_m = cm.source_extraction.cnmf.initialization.resize(cnmf_model_merged_m_refit.estimates.C, [K_m,T_m])
        if Cin_m.shape[1] < images2.shape[0]:
            Cin_m = np.concatenate((Cin_m,np.repeat(Cin_m[...,Cin_m.shape[1]-1][:,np.newaxis],images2.shape[0]-Cin_m.shape[1],axis=1)),axis=1,dtype=Cin_m.dtype)    
        YrAin_m = cm.source_extraction.cnmf.initialization.resize(cnmf_model_merged_m_refit.estimates.YrA, [K_m,T_m])
        if YrAin_m.shape[1] < images2.shape[0]:
            YrAin_m = np.concatenate((YrAin_m,np.repeat(YrAin_m[...,YrAin_m.shape[1]-1][:,np.newaxis],images2.shape[0]-YrAin_m.shape[1],axis=1)),axis=1,dtype=YrAin_m.dtype)  
        
        cnmf_model_mr_m_raw = cnmf.CNMF(n_processes, 
                               params=parameters, 
                               dview=dview)
        cnmf_model_mr_m_raw.estimates = deepcopy(cnmf_model_merged_m_refit.estimates)
        cnmf_model_mr_m_raw.estimates.C = Cin_m
        cnmf_model_mr_m_raw.estimates.YrA = YrAin_m
        cnmf_model_mr_m_raw.estimates.b = None
        cnmf_model_mr_m_raw.estimates.f = None
        cnmf_model_mr_m_raw.mmap_file = cnmf_model.mmap_file
        
        #~2.5 hrs on colab05, XX on beastmode, 5.5 hrs on megatron
        cnmf_model_mr_m_raw_refit = cnmf_model_mr_m_raw.refit(images2)
        print(cnmf_model_mr_m_raw_refit.estimates.C.shape)
        
        # save merged and refit outputs
        #print("Saving CNMF model...")
        save_path_mr_m_raw = os.path.join(data_dir, f'ch0_means_movie_nobg_cnmf_mr_m_raw.hdf5')
        save_path_mr_m_raw_refit = os.path.join(data_dir, f'ch0_means_movie_nobg_cnmf_mr_m_raw_refit.hdf5')
        cnmf_model_mr_m_raw.save(save_path_mr_m_raw)
        cnmf_model_mr_m_raw_refit.save(save_path_mr_m_raw_refit)
        
        print("Saved raw/refit models")

# STOP CLUSTER
cm.stop_server(dview=dview)

  0%|          | 0/2 [00:00<?, ?it/s]

D:\pkalugin\miniconda3\envs\caiman_full_102\lib\site-packages\scipy\signal\_spectral_py.py:600: UserWarning: nperseg = 256 is greater than input length  = 81, using nperseg = 81
  freqs, _, Pxy = _spectral_helper(x, y, fs, window, nperseg, noverlap,


*** Variable lam has not the same number of components as A ***
*** Variable SNR_comp has not the same number of components as A ***
(897, 81)
(266, 81)
(209, 81)
(209, 81)
Saved merged model
(5261760, 209, 9, <1x1 sparse matrix of type '<class 'numpy.float64'>'
	with 1 stored elements in Compressed Sparse Row format>)
(5261760, 209, 10, 76.5285609892527)
(5261760, 209, 0, 636371514.143957)
(5261760, 209, 11, 7464111.687489063)
(5261760, 209, 1, 5196756.7096202085)
(5261760, 209, 2, 5196756.7096202085)


D:\pkalugin\miniconda3\envs\caiman_full_102\lib\site-packages\scipy\signal\_spectral_py.py:600: UserWarning: nperseg = 256 is greater than input length  = 81, using nperseg = 81
  freqs, _, Pxy = _spectral_helper(x, y, fs, window, nperseg, noverlap,


(5261760, 209, 9, <1x1 sparse matrix of type '<class 'numpy.float64'>'
	with 1 stored elements in Compressed Sparse Row format>)
(5261760, 209, 10, 5196756.7096202085)
(5261760, 209, 0, 340379143.86307126)
(5261760, 209, 11, 7853630.569572617)
(5261760, 209, 1, 5593870.520376047)
(5261760, 209, 2, 5593870.520376047)


...

(149, 45)
Saved refit model
(5043555, 180, 9, <1x1 sparse matrix of type '<class 'numpy.float64'>'
	with 1 stored elements in Compressed Sparse Row format>)
(5043555, 180, 10, 45.37997905367039)
(5043555, 180, 0, 1188758236.6719434)
(5043555, 180, 11, 7856035.524563501)
(5043555, 180, 1, 5284605.8896233635)
(5043555, 180, 2, 5284605.8896233635)
(5043555, 180, 9, <1x1 sparse matrix of type '<class 'numpy.float64'>'
	with 1 stored elements in Compressed Sparse Row format>)
(5043555, 180, 10, 5284605.8896233635)
(5043555, 180, 0, 1143596637.4277244)
(5043555, 180, 11, 7823607.926592568)
(5043555, 180, 1, 5325440.046563723)
(5043555, 180, 2, 5325440.046563723)


D:\pkalugin\miniconda3\envs\caiman_full_102\lib\site-packages\scipy\sparse\_dia.py:277: RuntimeWarning: divide by zero encountered in remainder
  c = np.arange(num_rows, dtype=np.intc) - (offsets % max_dim)[:, None]


(180, 2340)
(5043555, 149, 9, <1x1 sparse matrix of type '<class 'numpy.float64'>'
	with 1 stored elements in Compressed Sparse Row format>)
(5043555, 149, 10, 54.83836441427904)
(5043555, 149, 0, 1962992911.3815367)
(5043555, 149, 11, 9553969.532907184)
(5043555, 149, 1, 6833592.456094297)
(5043555, 149, 2, 6833592.456094297)
(5043555, 149, 9, <1x1 sparse matrix of type '<class 'numpy.float64'>'
	with 1 stored elements in Compressed Sparse Row format>)
(5043555, 149, 10, 6833592.456094297)
(5043555, 149, 0, 1996958410.7132862)
(5043555, 149, 11, 9815713.304103326)
(5043555, 149, 1, 7040514.990207369)
(5043555, 149, 2, 7040514.990207369)


D:\pkalugin\miniconda3\envs\caiman_full_102\lib\site-packages\scipy\sparse\_dia.py:277: RuntimeWarning: divide by zero encountered in remainder
  c = np.arange(num_rows, dtype=np.intc) - (offsets % max_dim)[:, None]


(149, 2340)
Saved raw/refit models
